In [1]:
delete from [Theme_Park].[Maintenance_Calendar]
go
drop table [Theme_Park].[Maintenance_Notification]
go
create table [Theme_Park].[Maintenance_Notification]
(
    notification_id int identity(1,1),
    maintenance_id int not null,
    completion datetime not null,
    billed_hours int,
    amount smallmoney,
)

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the table 'Theme_Park.Maintenance_Notification', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.121

## Trigger TRIGGER_MANAGER_NOTIFICATION_ON_MAINTENANCE_COMPLETION:

This trigger is intended to create notification entries whenever a maintenance event is updated or inserted
with a completion time set. These notifications have 


In [1]:
drop trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
go
create trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
on [Theme_Park].[Maintenance_Calendar]
after insert,update
as
BEGIN
    set nocount on
    declare 
        @id int,
        @Completion_Date datetime,
        @Hours int,
        @Amount smallmoney,
        @Description varchar(50)
    declare cur cursor read_only for 
        select Maint_ID, Maint_Completion, Billed_Hours, Invoice_amount, Maint_Description from inserted
    open cur
    fetch next from cur into @id, @Completion_Date, @Hours, @Amount, @Description
    while @@fetch_status = 0
    begin
        if (@Completion_Date is not null)
        begin
            insert into [Theme_Park].[Maintenance_Notification]
            values (@id, @Completion_Date, @Hours, @Amount, @Description)
        end
        fetch next from cur into @id, @Completion_Date, @Hours, @Amount, @Description
    end
    close cur
    deallocate cur
end
    

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.156

In [4]:
insert into [Theme_Park].[Maintenance_Calendar]
(Occurence_datetime, Priority, Maint_Start, Maint_Completion, Billed_Hours, Invoice_amount)
values
(
    GETDATE(), 'ASAP', GETDATE(), GETDATE(), 1, 1130.50 
)

(1 row affected)

Total execution time: 00:00:00.061

In [8]:
insert into [Theme_Park].[Maintenance_Calendar]
(Occurence_datetime, Priority, Maint_Start)
values
(
    GETDATE(), 'ASAP', GETDATE()
)

(1 row affected)

Total execution time: 00:00:00.053

In [12]:
update [Theme_Park].[Maintenance_Calendar]
set Maint_Completion = GETDATE()
where Maint_ID = 3
go

(1 row affected)

Total execution time: 00:00:00.058

In [9]:
select * from [Theme_Park].[Maintenance_Notification]

(15 rows affected)

Total execution time: 00:00:00.061

notification_id,maintenance_id,completion,billed_hours,amount
1,1,2022-11-02 18:22:44.457,1,1130.50
2,2,2022-11-02 18:22:50.080,1,1130.50
3,3,2022-11-02 18:26:44.707,NULL,NULL
4,3,2022-11-02 18:26:53.157,NULL,NULL
5,13,2022-11-03 00:47:21.440,1,1.00
6,13,2022-11-03 00:48:10.653,1,1.00
7,13,2022-11-03 00:48:28.907,1,1.00
12,16,2022-11-03 00:00:00.000,5,5.00
15,21,2022-11-03 00:00:00.000,3,2.00
8,13,2022-11-03 00:48:48.937,1,1.00


In [16]:
declare @date datetime = GETDATE()
exec [Theme_Park].[Proc_Add_Maint] @date, null, null, 'Urgent', @date
go 

(1 row affected)

Total execution time: 00:00:00.086

In [8]:
select * from [Theme_Park].[Maintenance_Calendar]
go
select * from [Theme_Park].[Attractions]
GO
select * from [Theme_Park].[Vendor]
go
select * from [Theme_Park].[Department]
go
select * from [Theme_Park].[Ticket]
go
select * from [Theme_Park].[Merchandise_Sale]
go
select * from [Theme_Park].[Merchandise]
go
select * from [Theme_Park].[Employee]
go

(16 rows affected)

(1 row affected)

(1 row affected)

(4 rows affected)

(322 rows affected)

...

dept_id,dept_name,supervisor_ssn
3,Admin,NULL
4,Research,NULL
5,Food Court,NULL
6,Thrills,NULL


Ticket_ID,Date,Ticket_Class,Price,Reservation
1,2022-10-22 00:00:00.000,Poor,11.50,NULL
2,2022-10-22 00:00:00.000,Poor,11.50,NULL
3,2022-10-29 00:00:00.000,Poor,11.50,NULL
4,2022-12-01 00:00:00.000,Poor,11.50,NULL
5,2022-12-01 00:00:00.000,Premium,11.50,NULL
6,2022-10-31 00:00:00.000,Poor,11.50,NULL
7,2022-11-02 06:02:40.570,Normal,11.50,NULL
8,2022-11-02 06:02:58.347,Poor,11.50,NULL
13,2022-11-02 07:45:26.007,Poor,11.50,NULL
14,2022-11-02 07:45:38.123,Poor,11.50,NULL


transaction_id,item_id,quantity,sale_price,attraction_id,transaction_date
1,2,1,10.00,NULL,2022-11-15 02:26:13.000
2,5,8,8.00,NULL,2022-04-11 03:47:43.000
3,4,8,24.00,NULL,2022-06-24 01:55:10.000
4,5,1,1.00,NULL,2022-07-25 11:35:31.000
5,3,8,66.00,NULL,2022-01-04 12:10:51.000
6,2,6,60.00,NULL,2022-10-18 13:53:08.000
7,3,2,16.50,NULL,2022-05-23 11:08:57.000
8,4,1,3.00,NULL,2022-01-28 12:08:41.000
9,1,1,1.50,NULL,2022-05-18 21:37:05.000
10,1,1,1.50,NULL,2022-01-15 09:39:48.000


item_id,name,price,merch_type
1,Hot Dog,1.50,Food
2,Basic T-Shirt,10.00,Merch
3,Baseball Cap,8.25,Merch
4,Pizza Slice,3.00,Food
5,Soda,1.00,Food


fname,lname,ssn,gender,address,phone,date_joined,dept_id,role,supervisor_ssn,salaried,payrate,vacation_days
paul,george,666666666,F,main,5125125125,2022-10-11,3,operator,NULL,1,10000000.00,0.00
Phillip,Sudario,987654321,M,4800 Calhoun Rd,7137432255,2022-11-03,3,NULL,NULL,0,12.00,NULL
